vtp_dir & csvs_directory are set in ./data/vtp_to_point_cloud.py

mesh_resolution, cut_type, num_samples are set in ./data/DataLoad_utils.py

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Colab Notebooks/final'

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/final


In [2]:
!pip install vtk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 MB 12.4 MB/s eta 0:00:00


In [3]:
from pathlib import Path
checkpoints_dir = Path('./checkpoints/')
checkpoints_dir.mkdir(exist_ok=True)

In [4]:
from data.mesh_and_cut import *
from data.DataLoad_utils import validate_mesh_resolution,validate_cut_type
update_mesh_resolution(new_mesh_resolution=1)
update_cut_type(new_cut_type='ninja')
validate_mesh_resolution()
validate_cut_type()

mesh_resolution:5
cut_type:ninja
{'unruptured': 132, 'ruptured': 133}
{'unruptured': 263, 'ruptured': 87}
{'unruptured': 79, 'ruptured': 41}
['./data/original_data/remeshed/area-005/ninja/C0001_ninja.vtp', './data/original_data/remeshed/area-005/ninja/C0002_ninja.vtp', './data/original_data/remeshed/area-005/ninja/C0003_ninja.vtp', './data/original_data/remeshed/area-005/ninja/C0005_ninja.vtp', './data/original_data/remeshed/area-005/ninja/C0006_ninja.vtp', './data/original_data/remeshed/area-005/ninja/C0007a_ninja.vtp', './data/original_data/remeshed/area-005/ninja/C0008_ninja.vtp', './data/original_data/remeshed/area-005/ninja/C0009_ninja.vtp', './data/original_data/remeshed/area-005/ninja/C0010_ninja.vtp', './data/original_data/remeshed/area-005/ninja/C0011_ninja.vtp', './data/original_data/remeshed/area-005/ninja/C0012_ninja.vtp', './data/original_data/remeshed/area-005/ninja/C0013_ninja.vtp', './data/original_data/remeshed/area-005/ninja/C0014_ninja.vtp', './data/original_data/rem

In [5]:
from torch import load
from data.DataLoad_utils import mesh_resolution,cut_type,num_samples
from construct_PC_model import classifier
from tqdm import tqdm
from training_utils import judging_computation_device,scheduler,val,random_point_dropout,random_scale_point_cloud,shift_point_cloud,rotate_point_cloud
from construct_PC_model import optimizer,criterion,learning_rate
# import training_utils
from torch import Tensor, no_grad,save
import numpy as np
from data.DataLoad_utils import generate_dataset
from torch.utils.data import DataLoader

Pre-processing train data to ./data/processed_data/ready2feed_point_cloud/train/area_001_ninja_points_and_labels.dat
./data/processed_data/extracted_point_cloud/area-001/ninja/C0093_ninja.csv


FileNotFoundError: ignored

In [ ]:
device=judging_computation_device()

In [ ]:
try:
    checkpoint = load(f'./checkpoints/best_model_00{mesh_resolution}_{cut_type}.pth')
    start_epoch = checkpoint['epoch']
    classifier.load_state_dict(checkpoint['model_state_dict'])
    print('Paused model found. Resuming...')
except:
    print('No paused model, training from scratch...')
    start_epoch = 0

In [ ]:
set_epoch=100
global_epoch = 0

In [ ]:
global_step = 0

In [ ]:
classifier = classifier.to(device)
criterion = criterion.to(device)

In [ ]:
batch_size=9
num_workers=0
train_dataset,val_dataset,test_dataset=generate_dataset()
train_DataLoader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_DataLoader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_DataLoader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [ ]:
best_instance_acc = 0.0
best_class_acc = 0.0

In [ ]:
for epoch in range(start_epoch, set_epoch):
    print(f'Epoch {global_epoch+ 1} ({epoch+ 1}/{set_epoch}):')
    mean_correct = []#每个批次的准确率
    classifier = classifier.train()
    for batch_id, (points, target) in tqdm(enumerate(train_DataLoader, 0), total=len(train_DataLoader), smoothing=0.9):
        optimizer.zero_grad()#优化器梯度清零
        #数据增强
        points = points.data.numpy()
        points = random_point_dropout(points)#随机删除部分点
        points[:, :, 0:3] = random_scale_point_cloud(points[:, :, 0:3])#随机缩放
        points[:, :, 0:3] = shift_point_cloud(points[:, :, 0:3])#随机平移
        points[:, :, 0:3] = rotate_point_cloud(points[:, :, 0:3])#随机旋转
        points = Tensor(points)
        points = points.transpose(2, 1)#适应模型输入的格式

        points, target = points.to(device), target.to(device)

        pred, trans_feat = classifier(points)#预测值以及特征变换矩阵
        loss = criterion(pred, target.long(), trans_feat)#计算损失
        pred_choice = pred.data.max(1)[1]#每个样本的预测类别

        correct = pred_choice.eq(target.long().data).cpu().sum()#预测类别与真实标签相等的样本数
        mean_correct.append(correct.item() / float(points.size()[0]))#添加到 mean_correct 列表
        loss.backward()#计算并记录损失梯度
        optimizer.step()#通过优化器执行参数更新
        global_step += 1#记录总的训练步数
    scheduler.step()#更新学习率
    train_instance_acc = np.mean(mean_correct)#该epoch结束时计算并记录平均的训练实例准确率
    print(f'Train Instance Accuracy: f{train_instance_acc}')

    with no_grad():
        instance_acc, class_acc = val(classifier.eval(), val_DataLoader, num_class=2)#计算模型的实例准确率和类别准确率

        if instance_acc >= best_instance_acc:
            best_instance_acc = instance_acc
            best_epoch = epoch + 1

            print('Save checkpoint model status...')
            savepath = str(checkpoints_dir) + f'/best_model_00{mesh_resolution}_{cut_type}.pth'
            print('Saving at %s' % savepath)
            state = {
                'epoch': best_epoch,
                'instance_acc': instance_acc,
                'class_acc': class_acc,
                'model_state_dict': classifier.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }
            save(state, savepath)

        if class_acc >= best_class_acc:
            best_class_acc = class_acc
        print('Test Instance Accuracy: %f, Class Accuracy: %f' % (instance_acc, class_acc))
        print('Best Instance Accuracy: %f, Class Accuracy: %f' % (best_instance_acc, best_class_acc))

        global_epoch += 1